In [1]:
import pandas as pd
import numpy as np
import glob
path = r'/Users/satanfromhell/Desktop/Python/14-million-cell-phone-reviews' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0,encoding="ISO-8859-1")
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)




In [2]:
english = frame.loc[frame['lang']=='en']
english.extract.fillna('', inplace=True)
english= english.loc[english['extract']!='']
english.count()



/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


phone_url    550531
date         550531
lang         550531
country      550531
source       550531
domain       550531
score        545424
score_max    545424
extract      550531
author       548187
product      550531
dtype: int64

In [3]:
english.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [4]:
target=[]
phone_reviews = []
for i, r in english.iterrows():
    if(r.score>6):
        target.append(1)
        phone_reviews.append(r.extract)
    elif(r.score<4):
        target.append(0)
        phone_reviews.append(r.extract)
len(phone_reviews)
len(target)
    

463789

In [5]:
phone_reviews[1000]


'This is by far the best phone I have had. The only downfall is when you go to turn up or down the volume with one hand 9 out of 10 times my other finger turns the screen off. If the side buttons were moved to accomadate this issue there would be nothing else wrong with it.'

In [6]:
target.count(1)



385578

In [7]:
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", str(line).lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", str(line)) for line in reviews]
    return reviews

phone_reviews = preprocess_reviews(phone_reviews)
phone_reviews[1000]


'this is by far the best phone i have had the only downfall is when you go to turn up or down the volume with one hand 9 out of 10 times my other finger turns the screen off if the side buttons were moved to accomadate this issue there would be nothing else wrong with it'

In [8]:
#TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(binary=True,ngram_range=(1,2),stop_words=['and','did','over','the','since','at','rating','of','if','Samsung','processor','upgrade','product','phone','features'])
tfidf_vectorizer.fit(phone_reviews)
X = tfidf_vectorizer.transform(phone_reviews)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)


/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['samsung'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
#Linear SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score 
final_svm_ngram = LinearSVC(C=0.69)
final_svm_ngram.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_val, final_svm_ngram.predict(X_val)))




Final Accuracy: 0.9474419567392279


In [10]:
#Comparing the accuracy with Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression(C=0.79)
lr.fit(X_train, y_train)
print ("Accuracy: %s" 
           % accuracy_score(y_val,lr.predict(X_val)))
    



/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.9395591127056957
